In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request
import cv2
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [83]:
def Get_Locations(driverf, Name_list, stone_name):
    time.sleep(1)
    i = 0    
    loc_list = []
    loc_url = []
    final_loc = []
    driverf.find_element_by_css_selector("input[type='text']").send_keys(stone_name)
    time.sleep(1)
    try:
        driverf.find_element_by_css_selector("input[type='submit']").click()
    
    except ElementClickInterceptedException:
        try:
            driverf.find_element_by_css_selector("input[type='text']").send_keys(Keys.ENTER)
        except ElementClickInterceptedException:
            return loc_list
    
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    table = soup.find("table",{"class":"loclisttable"})
    for t in table:
        url_for_cords = t.find_all('a')
        for url in url_for_cords:
            href = url.get('href')
            loc_url.append(href)
            
        divs = t.find_all("div")
        for d in divs:
            loc_list.append(d.text)
            
            
    for loc in loc_url:
        if(loc.find('loc-')!=-1):
            http = "https://www.mindat.org/"+loc
            final_loc.append(http)
            
    return loc_list,final_loc



def save_photos(df):  
    for ind in df.index:
        i = 0
        image_url_list = df['Image_url'][ind]
        stone_name = df['Name'][ind]
        for image_url in image_url_list:
            FileName = "images.csv"
            FilePath = 'Data\Images\images'
            image_url = 'https://www.mindat.org/'+image_url
            response = requests.get(image_url)
            image_name = FilePath + 'mindat'+stone_name+'ind='+str(ind)+'num='+str(i) + '.jpg' 
            with open(image_name, 'wb') as file:
                file.write(response.content)
            i+=1
            img = cv2.imread(image_name)
    # cv2.imshow(img)



def get_photos(driverf,Name_list,stone_name):
    time.sleep(1)
    i = 0    
    img_list = []
    driverf.find_element_by_css_selector("input[type='text']").send_keys(stone_name)
    time.sleep(1)
    try:
        driverf.find_element_by_css_selector("input[type='submit']").click()
    
    except ElementClickInterceptedException:
        try:
            driverf.find_element_by_css_selector("input[type='text']").send_keys(Keys.ENTER)
        except ElementClickInterceptedException:
            return img_list
    
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    table = soup.find_all("div",{"class":"userbigpicture noborder"})
    for img in table:
        img_list.append(img.find('img')['src'])
        
        
    return img_list

In [84]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driverf = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://geology.com/gemstones/')
driverf.get('https://www.mindat.org/photoscroll.php')

soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find("div",{"class":"right"})
Name_list = []

# for ind in table:
#     href_list  = ind.find('a')['href']
href_list = table.find_all('a')
i = 0 ###
for href in href_list:
    data_dict = {}
    Name = href.text
    data_dict['Name'] = Name
    data_dict['Image_url'] = get_photos(driverf, Name_list, Name)
    loc1,loc2 = Get_Locations(driverf, Name_list, Name)
    data_dict['Locations'] = loc1
    data_dict['Locations_url'] = loc2
    Name_list.append(data_dict)
    i+=1###
    if(i==2):###
        break###

driver.close()
driverf.close()    
df = pd.DataFrame(Name_list)
# save_photos(df)    ################# להחזיר אחרי הבדיקות!!!!


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [85]:

df

,Name,Image_url,Locations,Locations_url
0,Amazonite,[/imagecache/11/7a/04766080014946246016426.jpg...,"[Argentina, , Australia, , Austria, , Brazi...","[https://www.mindat.org//loc-269722.html, http..."
1,Amethyst,[/imagecache/8a/ca/03039750014946246163278.jpg...,"[Afghanistan, , Angola, , Argentina, , Arme...","[https://www.mindat.org//loc-404929.html, http..."
